## Lab04
  
The [Conversation AI](https://conversationai.github.io/) team, a research initiative founded by [Jigsaw](https://jigsaw.google.com/) and Google (both a part of Alphabet) are working on tools to help improve online conversation. One area of focus is the study of negative online behaviors, like toxic comments (i.e. comments that are rude, disrespectful or otherwise likely to make someone leave a discussion).   
  
Kaggle are currently hosting their [second competition](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge#description) on this research. The challenge is to create a model that is capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. The competitions use a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

We shall be using this dataset to benchmark a number of ML models. While the focus of the current competition is to mitigate bias, we will not be using the metric used in the competition. Instead we will be focusing on a simpler metric [Area under the Curve (or AUC)](https://www.kaggle.com/learn-forum/53782) which is suitable to unbalanced binary datasets. Also, we shall not consider different levels of Toxicity; we shall purely take anything marked over the 0.5 level in the measured toxicity range as toxic, and anything underneath as non-toxic. 

We have created a jupyter notbook with some of the tools to model this problem in Deep Learning, using Logistic regression, MLP, CNN and RNNs. Your challenge will be to fill in the models and benchmark the accuracy you achieve on different models.

We shall be using the keras deep learning package. As you may know, this is an API into DL frameworks, but is most commonly backed by Tensorflow. [keras.io](keras.io) is a great source for documentation and examples on layers available andn functionality. 

**Have fun!!**


*Disclaimer: the dataset used contains text that may be considered profane, vulgar, or offensive.*

### Set up packages

In [ ]:
import os
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score

### Load data
Let's load our data and take a peak.   
The competition metrics and fields have too much detail too cover here.   
We will just focus on the comment of the users and whether it was deemed toxic (target>0.5) or not.    

In [ ]:
os.listdir()

In [ ]:
train_df = pd.read_csv('train.csv.zip', 
                       dtype={'comment_text':str},
                       usecols=['comment_text', 'target'],
                       compression = 'zip')
train_df['target'] = (train_df['target']>0.5).astype(int)
print("Dataframe Number of (rows, columns) : {}".format(train_df.shape))

In [ ]:
pd.options.display.max_colwidth = 200
train_df.head(10)

### Create validation data set
Lets randomly 66/33 split the data into a training and validation set.   
**No change needed here - note, please do not change the KFold split parameters, keeping it consistent will help us debug.**

In [ ]:
cv = KFold(n_splits=3, shuffle=True, random_state=42)
trn_ids, val_ids = next(cv.split(train_df))
x_train, x_valid = train_df['comment_text'][trn_ids], train_df['comment_text'][val_ids]
y_train, y_valid = train_df['target'].values[trn_ids], train_df['target'].values[val_ids]

### 1D CNN

Now we shall demonstrate how to apply 1D convolutions over sentences to learn trends.  
As shown in the diagram from wildml.com, the first step, is to create a vector representation (or embedding) of each word, or token, in the corpus. We do this by assigning a unique integer id to each token. We then create an array, where each row  is the vector for a single toke, whcih can be indexed by the unique id. This can be seen below keras's text tokenizer.  
![Image](http://www.wildml.com/wp-content/uploads/2015/11/Screen-Shot-2015-11-06-at-8.03.47-AM.png)  
Keras does not allow dynamic graphs, we need to define the graph in advance. To do this we need to define the length of the sequences. As the length of the sequences are fixed, we need to pad (or crop) each sentence to that fixed length.   
There is ways to get around this in Keras's which significantly increase speed, however, we do not cover that here. If you are interested, an example is [here](https://github.com/darraghdog/avito-demand/blob/d25c441e6c37557cb3ba1637df9487ca00b99822/nnet/nnet_2605.py#L230). 

In [ ]:
# We crop a lot of the texts by only making max length 50
MAX_LEN = 30
tokenizer = keras.preprocessing.text.Tokenizer() 
tokenizer.fit_on_texts(list(x_train) + list(x_valid))
word_index = tokenizer.word_index
X_trn_seq = tokenizer.texts_to_sequences(list(x_train))
X_val_seq = tokenizer.texts_to_sequences(list(x_valid))
X_trn_seq = keras.preprocessing.sequence.pad_sequences(X_trn_seq, maxlen=MAX_LEN)
X_val_seq = keras.preprocessing.sequence.pad_sequences(X_val_seq, maxlen=MAX_LEN)

Now we have a 2D array being fed in for each sentence sequence, which is the `<Embedding Vectors> * <Numer of tokens in the sequence>`. A sequence of 1D convolutions is applied to these matrices being inputted, along with max poolling. 

In [ ]:
inp = keras.Input(shape = (MAX_LEN,))
x = keras.layers.Embedding(len(X_trn_seq) + 1, 128)(inp)
x = keras.layers.SpatialDropout1D(0.3)(x)
x = keras.layers.Conv1D(64, 2, activation='relu', padding='same')(x)
x = keras.layers.MaxPooling1D(5, padding='same')(x)
x = keras.layers.Conv1D(64, 3, activation='relu', padding='same')(x)
x = keras.layers.MaxPooling1D(5, padding='same')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.1)(keras.layers.Dense(128, activation='relu') (x))
x = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inp, outputs=x)
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=1e-3), metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(X_trn_seq, y_train, batch_size=2**10, epochs=1, verbose=1, validation_data=(X_val_seq, y_valid))
preds_cnn = model.predict(X_val_seq).flatten()
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_cnn)))

### CNN with Embeddings
Embeddings are vector representations for words. Embeddings are covered in week 7, however we will check here if pretrained word embeddings our results. When learning from randomly generated embeddings, as seen above the model creates embeddings of the words based on the corpus in the training data only. Using pretrained embeddings, we can use embeddings of words pretrained on much larger datasets.  
  
For example, Stanford created the [Glove](https://nlp.stanford.edu/projects/glove/) created the `glove.840B.300d.txt` embedding vectors based on a Common Crawl of the web. It scanned 840Billion words (or tokens) and created embeddings for over 2Million tokens. Each token is represented by a vector of dimension 300 - ie. 300 floating point numbers to represent each token or word from the 2Million tokens found. 

**In this task we will load pretrained embeddings from disk into memory, where we assign each token an embedding. This is done by creating a numpy array of with a row for each index position in the `tokenizer` we created earlier. 
The width of the array will be the dimension of the vectors for each token.
Then you will load that embedding matrix into the Embedding layer in the keras model. As words are fed in from the array of tokenised sentences within the model, each token will be indexed into the embedding matrix to locate its equivalent embedding vector. Please set the embedding matrix to not trainable, so the pretrained embeddings do not change as the model learns.**  
**To understand how to load the pretrained embeddings into the embedding layer you can leverage the approach taken in [this script](https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout/data#Improved-LSTM-baseline) put together by Jeremy Howard to load the `glove.6B.50d.txt`.
Please do this by setting `trainable=False` in the embedding layer, when you have the pretrained embeddings loaded.**

In [ ]:
'''Follow Jeremy's approach to load one of the embedding files below 
Test with different embedding files. Remember, each files has a different embedding dimension.
https://www.kaggle.com/jhoward/improved-lstm-baseline-glove-dropout/data#Improved-LSTM-baseline
'''
[f for f in os.listdir() if 'glove' in f]

In [ ]:
inp = keras.Input(shape = (MAX_LEN,))
'''Students fill in how to load the embeddings from numpy array to Keras. 
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Again, follow Jeremy's approach to load the embeddings to the Embedding layer 
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
'''
x = keras.layers.SpatialDropout1D(0.3)(x)
x = keras.layers.Conv1D(64, 2, activation='relu', padding='same')(x)
x = keras.layers.MaxPooling1D(5, padding='same')(x)
x = keras.layers.Conv1D(64, 3, activation='relu', padding='same')(x)
x = keras.layers.MaxPooling1D(5, padding='same')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(0.1)(keras.layers.Dense(128, activation='relu') (x))
x = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inp, outputs=x)
model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(lr=1e-3), metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(X_trn_seq, y_train, batch_size=2**10, epochs=1, verbose=1, validation_data=(X_val_seq, y_valid))
preds_cnn = model.predict(X_val_seq).flatten()

In [ ]:
print('AUC score : {:.5f}'.format(roc_auc_score(y_valid, preds_cnn)))